# CB (Cornerback) round regression (Ridge)

Predict draft round 1–8 (8 = undrafted) using combine + PFF (Pass_Rush, Run_Defense, Pass_Coverage) + RAS, KNN imputation, Ridge regression.
- Train: 2015–2023 (cb_training.csv; RAS and PFF already merged in data_cleaning).
- Test: cb_testing.csv filtered to 2024/2025 (drafted only; actual rounds 1–7); 2026 from cb_drafted_2026.csv.

In [47]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# CB feature set: combine + RAS + PFF (run D: missed_tackle_rate, forced_fumbles; coverage) + p4_conference
FEATURES_WITH_COLLEGE_CB = [
    'Broad Jump', 'Vertical', '40yd', 'Height', 'Weight',
    'speed_score', 'explosive_score', 'RAS', 'arm_length_inches',
    'missed_tackle_rate', 'forced_fumbles',
    'yards_per_coverage_snap', 'forced_incompletion_rate', 'snap_counts_coverage', 'coverage_percent',
    'coverage_snaps_per_target', 'INT_rate', 'PBU_rate',
    'qb_rating_against', 'catch_rate', 'avg_depth_of_target',
    'p4_conference'
]
CONTAINS_WITH_COLLEGE_CB = [
    'contains_broad_jump', 'contains_vertical', 'contains_40yd', 'contains_height', 'contains_weight',
    'contains_speed_score', 'contains_explosive_score', 'contains_ras', 'contains_arm_length_inches',
    'contains_missed_tackle_rate', 'contains_forced_fumbles',
    'contains_yards_per_coverage_snap', 'contains_forced_incompletion_rate', 'contains_snap_counts_coverage', 'contains_coverage_percent',
    'contains_coverage_snaps_per_target', 'contains_INT_rate', 'contains_PBU_rate',
    'contains_qb_rating_against', 'contains_catch_rate', 'contains_avg_depth_of_target',
    'contains_p4_conference'
]
FEATURES_WITH_COLLEGE_ALL = FEATURES_WITH_COLLEGE_CB + CONTAINS_WITH_COLLEGE_CB

In [48]:
# Load CB training (2015–2023); RAS and PFF already in cb_training.csv from data_cleaning.py
df = pd.read_csv('../data/processed/cb_training.csv')
df = df[df['Year'].between(2015, 2023)].copy()
print('Train (2015–2023 Corners):', len(df))

Train (2015–2023 Corners): 312


In [49]:
# Print RAS and PFF availability
total_count = len(df)
ras_count = df['RAS'].notna().sum()
print(f"Players with RAS score: {ras_count} out of {total_count} ({ras_count/total_count*100:.1f}%)")
print(f"Players with missed_tackle_rate: {df['missed_tackle_rate'].notna().sum()} out of {total_count}")
print(f"Players with snap_counts_coverage: {df['snap_counts_coverage'].notna().sum()} out of {total_count}")

Players with RAS score: 201 out of 312 (64.4%)
Players with missed_tackle_rate: 273 out of 312
Players with snap_counts_coverage: 274 out of 312


In [50]:
# Height to inches
def height_inches(h):
    if pd.isna(h): return np.nan
    if isinstance(h, (int, float)) and not (isinstance(h, float) and np.isnan(h)):
        return float(h)
    s = str(h).strip()
    if '-' in s:
        parts = s.split('-')
        return int(parts[0]) * 12 + int(parts[1])
    return np.nan
df['Height'] = df['Height'].apply(height_inches)

# Speed score
df['speed_score'] = np.where(
    df['40yd'].notna() & (df['40yd'] > 0),
    df['Weight'] * 200 / (df['40yd'] ** 4),
    np.nan
)

# Explosive score (z-scores from this pool)
mean_v = df['Vertical'].mean()
std_v = df['Vertical'].std()
mean_b = df['Broad Jump'].mean()
std_b = df['Broad Jump'].std()
if std_v == 0 or np.isnan(std_v): std_v = 1.0
if std_b == 0 or np.isnan(std_b): std_b = 1.0
df['explosive_score'] = (df['Vertical'] - mean_v).fillna(0) / std_v + (df['Broad Jump'] - mean_b).fillna(0) / std_b

# P4 conference (hardcoded, same as DT)
school_alias = {
    'Ole Miss': 'Mississippi', 'Miami (FL)': 'Miami', 'Southern California': 'USC',
    'Central Florida': 'UCF', 'Brigham Young': 'BYU', 'Ohio St.': 'Ohio State',
    'Florida St.': 'Florida State', 'Kansas St.': 'Kansas State', 'Iowa St.': 'Iowa State',
    'Oklahoma St.': 'Oklahoma State', 'Penn St.': 'Penn State', 'San Diego St.': 'San Diego State',
    'San Jose St.': 'San Jose State', 'Boston Col.': 'Boston College', 'NC State': 'North Carolina State',
}
SEC_SCHOOLS = {'Alabama', 'Arkansas', 'Auburn', 'Florida', 'Georgia', 'Kentucky', 'LSU', 'Mississippi', 'Mississippi State', 'Missouri', 'South Carolina', 'Tennessee', 'Texas A&M', 'Vanderbilt', 'Oklahoma', 'Texas'}
BIG_TEN_SCHOOLS = {'Illinois', 'Indiana', 'Iowa', 'Maryland', 'Michigan', 'Michigan State', 'Minnesota', 'Nebraska', 'Northwestern', 'Ohio State', 'Penn State', 'Purdue', 'Rutgers', 'Wisconsin', 'UCLA', 'USC', 'Oregon', 'Washington'}
BIG_12_SCHOOLS = {'Baylor', 'Iowa State', 'Kansas', 'Kansas State', 'Oklahoma State', 'TCU', 'Texas Tech', 'West Virginia', 'BYU', 'UCF', 'Cincinnati', 'Houston', 'Arizona', 'Arizona State', 'Colorado', 'Utah'}
ACC_SCHOOLS = {'Boston College', 'Clemson', 'Duke', 'Florida State', 'Georgia Tech', 'Louisville', 'Miami', 'North Carolina', 'North Carolina State', 'NC State', 'Pittsburgh', 'Syracuse', 'Virginia', 'Virginia Tech', 'Wake Forest', 'California', 'SMU', 'Stanford'}
PAC12_SCHOOLS = {'Arizona', 'Arizona State', 'California', 'Colorado', 'Oregon', 'Oregon State', 'Stanford', 'UCLA', 'USC', 'Utah', 'Washington', 'Washington State'}
P4_SCHOOLS = SEC_SCHOOLS | BIG_TEN_SCHOOLS | BIG_12_SCHOOLS | ACC_SCHOOLS | PAC12_SCHOOLS
P4_SCHOOLS_NO_PAC12 = SEC_SCHOOLS | BIG_TEN_SCHOOLS | BIG_12_SCHOOLS | ACC_SCHOOLS

def is_p4(row):
    s = row.get('School')
    if pd.isna(s) or s == '': return 0
    sn = school_alias.get(s, s)
    year = row.get('Year', 2023)
    schools = P4_SCHOOLS if year <= 2023 else P4_SCHOOLS_NO_PAC12
    return 1 if sn in schools else 0
df['p4_conference'] = df.apply(is_p4, axis=1)

# Contains flags (all features)
df['contains_broad_jump'] = df['Broad Jump'].notna().astype(int)
df['contains_vertical'] = df['Vertical'].notna().astype(int)
df['contains_40yd'] = df['40yd'].notna().astype(int)
df['contains_height'] = df['Height'].notna().astype(int)
df['contains_weight'] = df['Weight'].notna().astype(int)
df['contains_speed_score'] = df['speed_score'].notna().astype(int)
df['contains_explosive_score'] = 1
df['contains_ras'] = df['RAS'].notna().astype(int)
df['contains_arm_length_inches'] = df['arm_length_inches'].notna().astype(int) if 'arm_length_inches' in df.columns else 0
df['contains_missed_tackle_rate'] = df['missed_tackle_rate'].notna().astype(int) if 'missed_tackle_rate' in df.columns else 0
df['contains_forced_fumbles'] = df['forced_fumbles'].notna().astype(int) if 'forced_fumbles' in df.columns else 0
df['contains_yards_per_coverage_snap'] = df['yards_per_coverage_snap'].notna().astype(int) if 'yards_per_coverage_snap' in df.columns else 0
df['contains_forced_incompletion_rate'] = df['forced_incompletion_rate'].notna().astype(int) if 'forced_incompletion_rate' in df.columns else 0
df['contains_snap_counts_coverage'] = df['snap_counts_coverage'].notna().astype(int) if 'snap_counts_coverage' in df.columns else 0
df['contains_coverage_percent'] = df['coverage_percent'].notna().astype(int) if 'coverage_percent' in df.columns else 0
df['contains_coverage_snaps_per_target'] = df['coverage_snaps_per_target'].notna().astype(int) if 'coverage_snaps_per_target' in df.columns else 0
df['contains_INT_rate'] = df['INT_rate'].notna().astype(int) if 'INT_rate' in df.columns else 0
df['contains_PBU_rate'] = df['PBU_rate'].notna().astype(int) if 'PBU_rate' in df.columns else 0
df['contains_qb_rating_against'] = df['qb_rating_against'].notna().astype(int) if 'qb_rating_against' in df.columns else 0
df['contains_catch_rate'] = df['catch_rate'].notna().astype(int) if 'catch_rate' in df.columns else 0
df['contains_avg_depth_of_target'] = df['avg_depth_of_target'].notna().astype(int) if 'avg_depth_of_target' in df.columns else 0
df['contains_p4_conference'] = df['School'].notna().astype(int)

In [51]:
# Target: round 1–7 if drafted, 8 if undrafted
y = np.where(
    df['Drafted'].astype(bool),
    np.clip(df['Round'].fillna(1).astype(int), 1, 7),
    8
)
X_raw = df[FEATURES_WITH_COLLEGE_ALL].copy()

# KNN imputation + scale
imputer = KNNImputer(n_neighbors=10)
X = imputer.fit_transform(X_raw)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Ridge regression
ridge = Ridge(alpha=1.0, random_state=42)
ridge.fit(X_scaled, y)

y_pred_train = np.clip(ridge.predict(X_scaled), 1, 8)
print('Train MAE (round 1–8):', round(mean_absolute_error(y, y_pred_train), 4))
print('Train samples:', len(y))

Train MAE (round 1–8): 1.791
Train samples: 312


In [52]:
def prepare_cb_df(ldf, year):
    """Add Height inches, speed_score, explosive_score, p4, contains_* to a CB dataframe (RAS/PFF already in CSV)."""
    ldf = ldf.copy()
    ldf['Year'] = year
    # Ensure columns needed for feature building exist (2026 CSV may lack Height/Weight)
    for col in ['Height', 'Weight', 'Vertical', 'Bench', 'Broad Jump', '3Cone', 'Shuttle', 'RAS']:
        if col not in ldf.columns:
            ldf[col] = np.nan
    if ldf['Height'].dtype == object or (ldf['Height'].astype(str).str.contains('-', na=False).any()):
        ldf['Height'] = ldf['Height'].apply(height_inches)
    else:
        ldf['Height'] = pd.to_numeric(ldf['Height'], errors='coerce')
    ldf['speed_score'] = np.where(
        ldf['40yd'].notna() & (ldf['40yd'] > 0),
        ldf['Weight'] * 200 / (ldf['40yd'] ** 4),
        np.nan
    )
    ldf['explosive_score'] = (ldf['Vertical'] - mean_v).fillna(0) / std_v + (ldf['Broad Jump'] - mean_b).fillna(0) / std_b
    ldf['p4_conference'] = ldf.apply(is_p4, axis=1)
    ldf['contains_broad_jump'] = ldf['Broad Jump'].notna().astype(int)
    ldf['contains_vertical'] = ldf['Vertical'].notna().astype(int)
    ldf['contains_40yd'] = ldf['40yd'].notna().astype(int)
    ldf['contains_height'] = ldf['Height'].notna().astype(int)
    ldf['contains_weight'] = ldf['Weight'].notna().astype(int)
    ldf['contains_speed_score'] = ldf['speed_score'].notna().astype(int)
    ldf['contains_explosive_score'] = 1
    ldf['contains_ras'] = ldf['RAS'].notna().astype(int)
    ldf['contains_arm_length_inches'] = ldf['arm_length_inches'].notna().astype(int) if 'arm_length_inches' in ldf.columns else 0
    ldf['contains_missed_tackle_rate'] = ldf['missed_tackle_rate'].notna().astype(int) if 'missed_tackle_rate' in ldf.columns else 0
    ldf['contains_forced_fumbles'] = ldf['forced_fumbles'].notna().astype(int) if 'forced_fumbles' in ldf.columns else 0
    ldf['contains_yards_per_coverage_snap'] = ldf['yards_per_coverage_snap'].notna().astype(int) if 'yards_per_coverage_snap' in ldf.columns else 0
    ldf['contains_forced_incompletion_rate'] = ldf['forced_incompletion_rate'].notna().astype(int) if 'forced_incompletion_rate' in ldf.columns else 0
    ldf['contains_snap_counts_coverage'] = ldf['snap_counts_coverage'].notna().astype(int) if 'snap_counts_coverage' in ldf.columns else 0
    ldf['contains_coverage_percent'] = ldf['coverage_percent'].notna().astype(int) if 'coverage_percent' in ldf.columns else 0
    ldf['contains_coverage_snaps_per_target'] = ldf['coverage_snaps_per_target'].notna().astype(int) if 'coverage_snaps_per_target' in ldf.columns else 0
    ldf['contains_INT_rate'] = ldf['INT_rate'].notna().astype(int) if 'INT_rate' in ldf.columns else 0
    ldf['contains_PBU_rate'] = ldf['PBU_rate'].notna().astype(int) if 'PBU_rate' in ldf.columns else 0
    ldf['contains_qb_rating_against'] = ldf['qb_rating_against'].notna().astype(int) if 'qb_rating_against' in ldf.columns else 0
    ldf['contains_catch_rate'] = ldf['catch_rate'].notna().astype(int) if 'catch_rate' in ldf.columns else 0
    ldf['contains_avg_depth_of_target'] = ldf['avg_depth_of_target'].notna().astype(int) if 'avg_depth_of_target' in ldf.columns else 0
    ldf['contains_p4_conference'] = ldf['School'].notna().astype(int)
    return ldf

# 2024 and 2025 from cb_testing.csv (PFF/RAS already merged in data_cleaning)
cb_testing = pd.read_csv('../data/processed/cb_testing.csv')
cb_2024 = prepare_cb_df(cb_testing[cb_testing['Year'] == 2024], 2024)
cb_2025 = prepare_cb_df(cb_testing[cb_testing['Year'] == 2025], 2025)

X_24_raw = cb_2024[FEATURES_WITH_COLLEGE_ALL].copy()
X_25_raw = cb_2025[FEATURES_WITH_COLLEGE_ALL].copy()
X_24 = imputer.transform(X_24_raw)
X_25 = imputer.transform(X_25_raw)
X_24_scaled = scaler.transform(X_24)
X_25_scaled = scaler.transform(X_25)

pred_24 = np.clip(ridge.predict(X_24_scaled), 1, 8)
pred_25 = np.clip(ridge.predict(X_25_scaled), 1, 8)

actual_24 = cb_2024['Round'].astype(int).values
actual_25 = cb_2025['Round'].astype(int).values

def eval_metrics(actual, pred, label):
    mae = mean_absolute_error(actual, pred)
    rmse = np.sqrt(mean_squared_error(actual, pred))
    r2 = r2_score(actual, pred)
    exact = (np.round(pred) == actual).mean()
    within_1 = (np.abs(np.round(pred) - actual) <= 1).mean()
    print(f'{label} (n={len(actual)}): MAE={mae:.4f}, RMSE={rmse:.4f}, R²={r2:.4f}, Exact={exact:.2%}, Within-1={within_1:.2%}')

print('2024 Corners:')
eval_metrics(actual_24, pred_24, '2024')
print('2025 Corners:')
eval_metrics(actual_25, pred_25, '2025')

2024 Corners:
2024 (n=36): MAE=1.3379, RMSE=1.6366, R²=0.3693, Exact=25.00%, Within-1=55.56%
2025 Corners:
2025 (n=19): MAE=1.7397, RMSE=2.0847, R²=-0.6275, Exact=21.05%, Within-1=47.37%


In [53]:
# Dataframes: players with actual round, model prediction, tier label, and interpretation
def pred_round_to_tier(p):
    if p < 1.75: return ('Round 1 Tier', 'True 1st-round grade')
    if p < 2.75: return ('Round 2 Tier', 'Early Day 2')
    if p < 3.75: return ('Round 3 Tier', 'Late Day 2')
    if p < 4.75: return ('Round 4 Tier', 'Early Day 3')
    if p < 5.75: return ('Round 5 Tier', 'Mid Day 3')
    if p < 6.75: return ('Round 6 Tier', 'Late Day 3')
    return ('Round 7 / UDFA Tier', 'Fringe draftable')

cb_2024_display = cb_2024[['Round', 'Pick', 'Player', 'School', 'Year']].copy()
cb_2024_display['predicted_round'] = pred_24
cb_2024_display['tier_label'] = [pred_round_to_tier(x)[0] for x in pred_24]
cb_2024_display['interpretation'] = [pred_round_to_tier(x)[1] for x in pred_24]
cb_2024_display['Round'] = cb_2024_display['Round'].astype(int)

cb_2025_display = cb_2025[['Round', 'Pick', 'Player', 'School', 'Year']].copy()
cb_2025_display['predicted_round'] = pred_25
cb_2025_display['tier_label'] = [pred_round_to_tier(x)[0] for x in pred_25]
cb_2025_display['interpretation'] = [pred_round_to_tier(x)[1] for x in pred_25]
cb_2025_display['Round'] = cb_2025_display['Round'].astype(int)

print('2024 drafted Corners')
display(cb_2024_display)
print('2025 drafted Corners')
display(cb_2025_display)

2024 drafted Corners


,Round,Pick,Player,School,Year,predicted_round,tier_label,interpretation
0,1,24.0,Terrion Arnold,Alabama,2024,3.057076,Round 3 Tier,Late Day 2
1,1,22.0,Quinyon Mitchell,Toledo,2024,2.864584,Round 3 Tier,Late Day 2
2,2,40.0,Cooper DeJean,Iowa,2024,2.232683,Round 2 Tier,Early Day 2
3,2,41.0,Kool-Aid McKinstry,Alabama,2024,3.371141,Round 3 Tier,Late Day 2
4,1,30.0,Nate Wiggins,Clemson,2024,2.714363,Round 2 Tier,Early Day 2
5,2,42.0,Kamari Lassiter,Georgia,2024,4.955873,Round 5 Tier,Mid Day 3
6,5,130.0,T.J. Tampa,Iowa State,2024,4.520814,Round 4 Tier,Early Day 3
7,2,43.0,Max Melton,Rutgers,2024,3.559563,Round 3 Tier,Late Day 2
8,2,50.0,Mike Sainristil,Michigan,2024,4.236617,Round 4 Tier,Early Day 3
9,5,140.0,Cam Hart,Notre Dame,2024,3.865468,Round 4 Tier,Early Day 3


2025 drafted Corners


,Round,Pick,Player,School,Year,predicted_round,tier_label,interpretation
36,1,20.0,Jahdae Barron,Texas,2025,2.560933,Round 2 Tier,Early Day 2
37,2,47.0,Will Johnson,Michigan,2025,5.367231,Round 5 Tier,Mid Day 3
38,1,30.0,Maxwell Hairston,Kentucky,2025,4.284723,Round 4 Tier,Early Day 3
39,2,61.0,Trey Amos,Ole Miss,2025,3.070696,Round 3 Tier,Late Day 2
40,2,53.0,Benjamin Morrison,Notre Dame,2025,5.335504,Round 5 Tier,Mid Day 3
41,3,68.0,Darien Porter,Iowa State,2025,1.325044,Round 1 Tier,True 1st-round grade
42,3,76.0,Shavon Revel Jr,East Carolina,2025,5.978363,Round 6 Tier,Late Day 3
43,4,131.0,Quincy Riley,Louisville,2025,4.429220,Round 4 Tier,Early Day 3
44,6,177.0,Dorian Strong,Virginia Tech,2025,4.810570,Round 5 Tier,Mid Day 3
45,3,84.0,Jacob Parrish,Kansas State,2025,3.721456,Round 3 Tier,Late Day 2


In [54]:
# 2026 evaluation (cb_drafted_2026.csv has PFF/RAS from data_cleaning)
cb_2026 = prepare_cb_df(pd.read_csv('cb_drafted_2026.csv'), 2026)

X_26_raw = cb_2026[FEATURES_WITH_COLLEGE_ALL].copy()
X_26 = imputer.transform(X_26_raw)
X_26_scaled = scaler.transform(X_26)

pred_26 = np.clip(ridge.predict(X_26_scaled), 1, 8)

# Check if Round column exists and has valid values for evaluation
if 'Round' in cb_2026.columns and cb_2026['Round'].notna().any():
    actual_26 = cb_2026['Round'].astype(int).values
    print('2026 Corners:')
    eval_metrics(actual_26, pred_26, '2026')
else:
    print(f'2026 Corners (n={len(pred_26)}): Predictions generated (no actual rounds available)')

# Display 2026 predictions
cb_2026_display = cb_2026[['Round', 'Pick', 'Player', 'School', 'Year']].copy()
cb_2026_display['predicted_round'] = pred_26
cb_2026_display['tier_label'] = [pred_round_to_tier(x)[0] for x in pred_26]
cb_2026_display['interpretation'] = [pred_round_to_tier(x)[1] for x in pred_26]
if 'Round' in cb_2026_display.columns and cb_2026_display['Round'].notna().any():
    cb_2026_display['Round'] = cb_2026_display['Round'].astype(int)

print('\n2026 Corners (predictions)')
display(cb_2026_display)

2026 Corners (n=24): Predictions generated (no actual rounds available)

2026 Corners (predictions)


,Round,Pick,Player,School,Year,predicted_round,tier_label,interpretation
0,NaN,NaN,Mansoor Delane,Virginia Tech,2026,1.943159,Round 2 Tier,Early Day 2
1,NaN,NaN,Jermod McCoy,Oregon State,2026,3.832851,Round 4 Tier,Early Day 3
2,NaN,NaN,Avieon Terrell,Clemson,2026,3.691317,Round 3 Tier,Late Day 2
3,NaN,NaN,Colton Hood,Auburn,2026,5.087809,Round 5 Tier,Mid Day 3
4,NaN,NaN,Brandon Cisse,North Carolina State,2026,5.702227,Round 5 Tier,Mid Day 3
5,NaN,NaN,Chris Johnson,San Diego State,2026,3.931722,Round 4 Tier,Early Day 3
6,NaN,NaN,Keith Abney II,Arizona State,2026,4.421624,Round 4 Tier,Early Day 3
7,NaN,NaN,D'Angelo Ponds,James Madison,2026,5.946795,Round 6 Tier,Late Day 3
8,NaN,NaN,Keionte Scott,Auburn,2026,4.403506,Round 4 Tier,Early Day 3
9,NaN,NaN,Malik Muhammad,Texas,2026,4.808248,Round 5 Tier,Mid Day 3


In [55]:
# Model results on entire training set (2017–2023), ordered by predicted_round
train_display = df[['Round', 'Pick', 'Player', 'School', 'Year']].copy()
train_display['predicted_round'] = y_pred_train
train_display['tier_label'] = [pred_round_to_tier(x)[0] for x in y_pred_train]
train_display['interpretation'] = [pred_round_to_tier(x)[1] for x in y_pred_train]
train_display = train_display.sort_values('predicted_round').reset_index(drop=True)
train_display

,Round,Pick,Player,School,Year,predicted_round,tier_label,interpretation
0,1.0,11.0,Marshon Lattimore,Ohio State,2017,1.481685,Round 1 Tier,True 1st-round grade
1,1.0,4.0,Denzel Ward,Ohio State,2018,1.859372,Round 2 Tier,Early Day 2
2,1.0,27.0,Byron Jones,Connecticut,2015,2.051433,Round 2 Tier,Early Day 2
3,1.0,24.0,Gareon Conley,Ohio State,2017,2.192128,Round 2 Tier,Early Day 2
4,3.0,94.0,Jamel Dean,Auburn,2019,2.196668,Round 2 Tier,Early Day 2
...,...,...,...,...,...,...,...,...
307,NaN,NaN,Ryan Pulley,Arkansas,2019,8.000000,Round 7 / UDFA Tier,Fringe draftable
308,NaN,NaN,Cameron Brown,Ohio St.,2023,8.000000,Round 7 / UDFA Tier,Fringe draftable
309,NaN,NaN,Henre Toliver,Arkansas,2018,8.000000,Round 7 / UDFA Tier,Fringe draftable
310,NaN,NaN,Corey Straughter,La-Monroe,2021,8.000000,Round 7 / UDFA Tier,Fringe draftable
